# ItaData2024

## Bronchiolitis

propositional analysis

In [27]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Settings

In [28]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

features = :catch9
# features = :minmax
# features = :custom

loadset = false
# loadset = true

scale = :semitones
# scale = :mel_htk

usemfcc = false
# usemfcc = true
usef0 = false
# usef0 = true

sr = 8000
audioparams = (
    usemfcc = usemfcc,
    usef0 = usef0,
    sr = sr,
    nfft = 512,
    scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    nbands = scale == :semitones ? 14 : 26,
    ncoeffs = scale == :semitones ? 7 : 13,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = usemfcc ? false : true,
)

memguard = false;
# memguard = true;
n_elems = 15;

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]

@assert experiment in avail_exp "Unknown type of experiment: $experiment."

findhealthy = y -> findall(x -> x == "Healthy", y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
findsick = y -> findall(x -> x == String(experiment), y)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"
memguard && begin filename *= string("_memguard") end

destpath = "results/propositional"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

### Audio features extraction function
This function is called for every audio sample and extract 25 features:
14 bands of the mel spectrogram,
11 spectral features: centroid, crest, entropy, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [29]:
function afe(x::AbstractVector{Float64}, audioparams::NamedTuple; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = audioparams.sr
    norm = true
    speech_detection = false
    # stft module
    nfft = audioparams.nfft
    win_type = (:hann, :periodic)
    win_length = audioparams.nfft
    overlap_length = round(Int, audioparams.nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = audioparams.nbands
    scale = audioparams.scale               # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = audioparams.freq_range
    # mel spectrogram module
    db_scale = audioparams.db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    if audioparams.usemfcc
        # mfcc module
        ncoeffs = audioparams.ncoeffs
        rectification = :log                    # :log, :cubic_root
        dither = true

        mfcc = get_mfcc(
            source=melspec,
            ncoeffs=ncoeffs,
            rectification=rectification,
            dither=dither,
        );
    end

    if audioparams.usef0
        # f0 module
        method = :nfc
        f0_range = (50, 400)

        f0 = get_f0(
            source=stftspec,
            method=method,
            freq_range=f0_range
        );
    end

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        filter(!isnothing, [
            melspec.spec',
            audioparams.usemfcc ? mfcc.mfcc' : nothing,
            audioparams.usef0 ? f0.f0 : nothing,
            spect.centroid,
            spect.crest,
            spect.entropy,
            spect.flatness,
            spect.flux,
            spect.kurtosis,
            spect.rolloff,
            spect.skewness,
            spect.decrease,
            spect.slope,
            spect.spread
        ])...
    )    
end

afe (generic function with 2 methods)

### Prepare dataset for analysis

In [30]:
d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

memguard && begin
    cat2 = round(Int, length(y)/2)
    indices = [1:n_elems; cat2:cat2+n_elems-1]
    x = x[indices, :]
    y = y[indices]
end

freq = round.(Int, afe(x[1, :audio], audioparams; get_only_melfreq=true))

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.nbands],
        audioparams.usemfcc ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.ncoeffs] : String[],
        audioparams.usef0 ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)
    
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

if !loadset
    @info("Build dataset...")

    X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])
    audiofeats = [afe(row[:audio], audioparams) for row in eachrow(x)]
    push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audiofeats])...)

    yc = CategoricalArray(y);

    train_ratio = 0.8

    train, test = partition(eachindex(yc), train_ratio, shuffle=true)
    X_train, y_train = X[train, :], yc[train]
    X_test, y_test = X[test, :], yc[test]

    println("Training set size: ", size(X_train), " - ", length(y_train))
    println("Test set size: ", size(X_test), " - ", length(y_test))
end

┌ Info: Build dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:42


Training set size: (222, 225) - 222
Test set size: (56, 225) - 56


### Train a model

In [31]:
if !loadset
    learned_dt_tree = begin
        Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
        model = Tree(max_depth=-1, )
        mach = machine(model, X_train, y_train)
        fit!(mach)
        fitted_params(mach).tree
    end
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


min(flatn) < 0.0008796
├─ min(mel3=504Hz) < -5.956
│  ├─ Healthy (3/3)
│  └─ min(rllff) < 648.4
│     ├─ Bronchiolitis (50/50)
│     └─ Healthy (2/2)
└─ std(entrp) < 0.09744
   ├─ max(sprd) < 845.8
   │  ├─ min(flatn) < 0.01135
   │  │  ├─ qnt(slope) < 1.933
   │  │  │  ├─ Healthy (27/27)
   │  │  │  └─ std(mel2=424Hz) < 0.6111
   │  │  │     ⋮
   │  │  │     
   │  │  └─ min(mel3=504Hz) < -4.259
   │  │     ├─ Bronchiolitis (6/6)
   │  │     └─ Healthy (1/1)
   │  └─ Healthy (31/31)
   └─ med(sprd) < 674.1
      ├─ std(kurts) < 45.22
      │  ├─ mean(mel1=357Hz) < -3.754
      │  │  ├─ Healthy (4/4)
      │  │  └─ max(slope) < -3.638e-7
      │  │     ⋮
      │  │     
      │  └─ 3ac(flatn) < 0.02418
      │     ├─ Bronchiolitis (4/4)
      │     └─ med(mel13=2832Hz) < -5.993
      │        ⋮
      │        
      └─ Healthy (13/13)


### Model inspection & rule study

In [32]:
if !loadset
    sole_dt = solemodel(learned_dt_tree)
    # Make test instances flow into the model, so that test metrics can, then, be computed.
    apply!(sole_dt, X_test, y_test);
    # Save solemodel to disk
    jldsave(jld2file, true; sole_dt)
else
    @info("Load dataset...")
    d = jldopen(jld2file)
    sole_dt = d["sole_dt"]
    close(d)
end
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ min(flatn) < 0.0008796058763419051
├✔ min(mel3=504Hz) < -5.956346010161386
│├✔ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│└✘ min(rllff) < 648.4375
│ ├✔ Bronchiolitis : (ninstances = 18, ncovered = 18, confidence = 0.89, lift = 1.0)
│ └✘ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
└✘ std(entrp) < 0.09743958981966608
 ├✔ max(sprd) < 845.7959454430622
 │├✔ min(flatn) < 0.01135037096621694
 ││├✔ qnt(slope) < 1.9334308535014981
 │││├✔ Healthy : (ninstances = 6, ncovered = 6, confidence = 0.83, lift = 1.0)
 │││└✘ std(mel2=424Hz) < 0.6111157235955555
 │││ ├✔ bsd(cntrd) < 2.5
 │││ │├✔ Bronchiolitis : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
 │││ │└✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 0.0, lift = NaN)
 │││ └✘ Bronchiolitis : (ninstances = 1, ncovered = 1, confidence = 0.0, lift = NaN)
 ││└✘ min(mel3=504Hz) < -4.259348335008356
 ││ ├✔ Bronchiolitis : (ninstances = 0, ncovered = 0, confidence = NaN, l

### Extract rules that are at least as good as a random baseline model

In [33]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (min(flatn) < 0.0008796058763419051) ∧ (¬(min(mel3=504Hz) < -5.956346010161386)) ∧ (min(rllff) < 648.4375)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 18, coverage = 0.32, confidence = 0.89, lift = 1.56, natoms = 3)
▣ (¬(min(flatn) < 0.0008796058763419051)) ∧ (std(entrp) < 0.09743958981966608) ∧ (max(sprd) < 845.7959454430622) ∧ (min(flatn) < 0.01135037096621694) ∧ (qnt(slope) < 1.9334308535014981)  ↣  Healthy : (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 0.83, lift = 1.94, natoms = 5)
▣ (¬(min(flatn) < 0.0008796058763419051)) ∧ (std(entrp) < 0.09743958981966608) ∧ (¬(max(sprd) < 845.7959454430622))  ↣  Healthy : (ninstances = 56, ncovered = 8, coverage = 0.14, confidence = 1.0, lift = 2.33, natoms = 3)
▣ (¬(min(flatn) < 0.0008796058763419051)) ∧ (¬(std(entrp) < 0.09743958981966608)) ∧ (med(sprd) < 674.121219794237) ∧ (std(kurts) < 45.21876382372958) ∧ (¬(mean(mel1=357Hz) < -3.7539557770423526)) ∧ (¬(max(slope) < -3.638154392506077e-7))  ↣  Bronchiolitis : (nin

### Simplify rules while extracting and prettify result

In [34]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (min(flatn) < 0.0) ∧ (min(mel3=504Hz) ≥ -5.96) ∧ (min(rllff) < 648.44)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 18, coverage = 0.32, confidence = 0.89, lift = 1.56, natoms = 3)
▣ (min(flatn) ∈ [0.0,0.01)) ∧ (std(entrp) < 0.1) ∧ (max(sprd) < 845.8) ∧ (qnt(slope) < 1.93)  ↣  Healthy : (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 0.83, lift = 1.94, natoms = 4)
▣ (min(flatn) ≥ 0.0) ∧ (std(entrp) < 0.1) ∧ (max(sprd) ≥ 845.8)  ↣  Healthy : (ninstances = 56, ncovered = 8, coverage = 0.14, confidence = 1.0, lift = 2.33, natoms = 3)
▣ (min(flatn) ≥ 0.0) ∧ (std(entrp) ≥ 0.1) ∧ (med(sprd) < 674.12) ∧ (std(kurts) < 45.22) ∧ (mean(mel1=357Hz) ≥ -3.75) ∧ (max(slope) ≥ -0.0)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 11, coverage = 0.2, confidence = 0.82, lift = 1.43, natoms = 6)
▣ (min(flatn) ≥ 0.0) ∧ (std(entrp) ≥ 0.1) ∧ (med(sprd) < 674.12) ∧ (std(kurts) ≥ 45.22) ∧ (3ac(flatn) ≥ 0.02) ∧ (med(mel13=2832Hz) < -5.99) ∧ (mean(mel1=357Hz) ≥ -4.3)  ↣  Bronchiolitis : (ni

### Directly access rule metrics

In [35]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

6-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 56, ncovered = 18, coverage = 0.32142857142857145, confidence = 0.8888888888888888, lift = 1.5555555555555556, natoms = 3)
 (ninstances = 56, ncovered = 6, coverage = 0.10714285714285714, confidence = 0.8333333333333334, lift = 1.9444444444444446, natoms = 5)
 (ninstances = 56, ncovered = 8, coverage = 0.14285714285714285, confidence = 1.0, lift = 2.3333333333333335, natoms = 3)
 (ninstances = 56, ncovered = 11, coverage = 0.19642857142857142, confidence = 0.8181818181818182, lift = 1.4318181818181819, natoms = 6)
 (ninstances = 56, ncovered = 1, coverage = 0.017857142857142856, confidence = 1.0, lift = 1.75, natoms = 7)
 (ninstances = 56, ncovered = 4, coverage = 0.07142857142857142, confidence = 1.0, lift = 2.3333333333333335, natoms = 3)

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [36]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (min(flatn) ≥ 0.0008796058763419051) ∧ (std(entrp) ≥ 0.09743958981966608) ∧ (med(sprd) < 674.121219794237) ∧ (std(kurts) ≥ 45.21876382372958) ∧ (3ac(flatn) ≥ 0.024179998538972895) ∧ (med(mel13=2832Hz) < -5.993137615945026) ∧ (mean(mel1=357Hz) ≥ -4.302643444322027)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 1, coverage = 0.017857142857142856, confidence = 1.0, lift = 1.75, natoms = 7, height = 6)
▣ (min(flatn) ≥ 0.0008796058763419051) ∧ (std(entrp) ≥ 0.09743958981966608) ∧ (med(sprd) ≥ 674.121219794237)  ↣  Healthy : (ninstances = 56, ncovered = 4, coverage = 0.07142857142857142, confidence = 1.0, lift = 2.3333333333333335, natoms = 3, height = 2)
▣ (min(flatn) ∈ [0.0008796058763419051,0.01135037096621694)) ∧ (std(entrp) < 0.09743958981966608) ∧ (max(sprd) < 845.7959454430622) ∧ (qnt(slope) < 1.9334308535014981)  ↣  Healthy : (ninstances = 56, ncovered = 6, coverage = 0.10714285714285714, confidence = 0.8333333333333334, lift = 1.9444444444444446, natoms = 4, height = 3)
▣ (min(

### Pretty table of rules and their metrics

In [37]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬────────────┬──────────┬───────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                               Antecedent │    Consequent │ ninstances │ ncovered │  coverage │ confidence │    lift │ natoms │ height │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────

# Inspect features

In [38]:
interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ (min(flatn) < 0.0) ∧ (min(mel3=504Hz) ≥ -5.96) ∧ (min(rllff) < 648.44)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 18, coverage = 0.32, confidence = 0.89, lift = 1.56, natoms = 3)
▣ (min(flatn) ∈ [0.0,0.01)) ∧ (std(entrp) < 0.1) ∧ (max(sprd) < 845.8) ∧ (qnt(slope) < 1.93)  ↣  Healthy : (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 0.83, lift = 1.94, natoms = 4)
▣ (min(flatn) ≥ 0.0) ∧ (std(entrp) < 0.1) ∧ (max(sprd) ≥ 845.8)  ↣  Healthy : (ninstances = 56, ncovered = 8, coverage = 0.14, confidence = 1.0, lift = 2.33, natoms = 3)
▣ (min(flatn) ≥ 0.0) ∧ (std(entrp) ≥ 0.1) ∧ (med(sprd) < 674.12) ∧ (std(kurts) < 45.22) ∧ (mean(mel1=357Hz) ≥ -3.75) ∧ (max(slope) ≥ -0.0)  ↣  Bronchiolitis : (ninstances = 56, ncovered = 11, coverage = 0.2, confidence = 0.82, lift = 1.43, natoms = 6)


In [39]:
interesting_features = unique(SoleData.feature.(SoleLogics.value.(vcat(SoleLogics.atoms.(i.antecedent for i in interesting_rules)...))))
interesting_variables = sort(SoleData.i_variable.(interesting_features))

10-element Vector{Symbol}:
 Symbol("max(slope)")
 Symbol("max(sprd)")
 Symbol("mean(mel1=357Hz)")
 Symbol("med(sprd)")
 Symbol("min(flatn)")
 Symbol("min(mel3=504Hz)")
 Symbol("min(rllff)")
 Symbol("qnt(slope)")
 Symbol("std(entrp)")
 Symbol("std(kurts)")